In [24]:
#PigLab3NoteBook

! pip install pandas
! pip install openpyxl
!pip install rdflib
import pandas as pd
import rdflib
import hashlib
import numpy as np
import urllib.parse
from rdflib import Literal, Namespace, RDF, URIRef, Graph, RDFS, OWL
from rdflib.namespace import FOAF, XSD
from rdflib.plugins.sparql import prepareQuery
from pyspark.sql.functions import when, col, lit

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


### Step 01: Define the file paths and Load the data into Pandas dataframes and Clean

In [25]:
# Define the file paths
files = ['data/PigData/Lab3/Copia de Base Datos DECIDE-INFLUENZA.xlsx']

# Load the data into Pandas dataframes
dfs = []
for file in files:
    df = pd.read_excel(file, engine='openpyxl')
    dfs.append(df)

df = dfs[0]


In [26]:
df

,ID,PROVINCIA,FECHA,PATOGENO,RESULTADO,MUESTRA,EDAD,GRANJA
0,SV210020,Murcia,05/01/2021,Influenza,Negativo,Hisopos,Cerdas abortadas,Granja de madres
1,210031,Badajoz,05/01/2021,Influenza,Negativo,Pulmon,NaN,Cebadero
2,210071,Girona,08/01/2021,Influenza,Positivo,Fluidos orales,NaN,Cebadero
3,210140,Salamanca,14/01/2021,Influenza,Negativo,Pulmon,NaN,Cebadero
4,210141,Salamanca,14/01/2021,Influenza,Negativo,Pulmon,2.5 meses,Cebadero
...,...,...,...,...,...,...,...,...
283,234707,Castellón,17/10/2023,Influenza,Positivo,Lenguas,NaN,Granja de madres
284,SV232530,Murcia,18/10/2023,Influenza,Negativo,Pulmon,Cebo,Cebadero
285,234780,NaN,20/10/2023,Influenza,Positivo,Hisopos nasales,NaN,Cebadero
286,SV232553,Zamora,20/10/2023,Influenza,Negativo,Pulmones,3.5 - 4 meses,Cebadero


In [27]:
#Rename Colums

df = df.rename(columns={
    'ID': 'SampleNumber',
    'PROVINCIA': 'Province',
    'FECHA': 'Date',
    'PATOGENO': 'Pathogen',
    'RESULTADO': 'Result',
    'MUESTRA': 'SampleType',
    'EDAD': 'Age', #Age of animal
    'GRANJA': 'FarmType',
})

# Set DiagnosticTest to "PCR" Country to "Spain", Breed to "Meat", and LabReference to "1".
df['DiagnosticTest'] = 'PCR'
df['Country'] = 'Spain'
df['Breed'] = 'Meat'
df["Lab_Reference"] = "3"




In [28]:
#Annotation Mapping

# Add Result mapping for NEG = 0 and POS =1
resultMapping = {
    "Positivo": "1",
    "Negativo": "0",
}
df["Result"] = df["Result"].map(resultMapping).fillna("Missing")

PathogenMapping = {
    "M. hyopneumoniae": "Maycoplasma hyopneumoniae",
    "Mycoplasma hyopneumoniae antistoffen" : "Maycoplasma hyopneumoniae",
    "Mycoplasma hyopneumoniae" : "Maycoplasma hyopneumoniae",
    "Influenza" : "Swine influenza virus",
    "Lawsonia intracellularis": "Lawsonia intracellularis",
    
}

# SampleTypeMapping for the additional sample types
SampleTypeMapping = {
    "Pulmón": "Tissue",
    "Pulmon": "Tissue",
    "Pulmones": "Tissue",
    "Lechón": "Tissue",
    "Lechon": "Tissue",
    "Lechones": "Tissue",
    "Lenguas lechones": "Tissue",
    "Lenguas": "Tissue",
    "Fluido oral": "Fluid",
    "Fluidos orales": "Fluid",
    "Pulmón/ Fluido oral": "Mixed",
    "Heces": "Feces",
    "Raspado traqueal": "Swab",
    "Hisopo/ Raspado traqueal/ Pulmón": "Mixed",
    "Intestino": "Tissue",
    "Hisopo": "Swab",
    "Hisopos": "Swab",
    "Hisopos nasales": "Swab",
    "Raspado traqueal/ Pulmón": "Mixed",
    "Fluido oral/ Pulmón": "Mixed",
    "Heces/intestino": "Feces",
    "Hisopo rectal": "Swab",
    "Porción intestinal": "Tissue",
    "Exudado": "Fluid",
    "Pulmon y ganglios ": "Tissue",
    "Líquido pericárdico": "Fluid",
    "Raspado traqueal/ Fluido oral": "Mixed",
    "Heces/hisopo rectal": "Feces",
    "Hisopo nasal": "Swab",
    "Hisopo nasal/ Pulmón/ Fluido Oral": "Mixed",
    " Lechones/hisopos nasales": "Mixed",
    "Órgano": "Tissue",
    "Hisopo intestinal": "Swab",
    "Feto": "Tissue",
    "Fetos": "Tissue",
    "Cabeza, cuatro patas y pulmon": "Swab",
    "Hisopo pulmonar": "Swab",
    "Heces/hisopos": "Feces",
    "Ganglio": "Tissue",
    "Intestino/heces/hisopo": "Mixed",
    "Pulmón/ Fluido Oral": "Mixed",
    "Pulmón / Fluido Oral": "Mixed",
    "Pulmón /gánglios e hisopos": "Mixed",
    "Hisopo fecal": "Swab",
    "Hisopos de vias respiratorias": "Swab",
    "Cabeza": "Tissue",
    "Feto y placenta": "Tissue",
    "Lenguas y cordones umbilicales": "Tissue",
    "Pulmnes + 1 lechon": "Tissue",
    "Sueros": "Blood",
    'Hisopos': 'Swab',
    'Pulmon': 'Tissue',
    'Fluidos orales': 'Fluid',
    'Lechon': 'Tissue',
    'Pulmones': 'Tissue',
    'Pulmon y ganglios': 'Mixed',
    'Lechones': 'Tissue',
    'Hisopo': 'Swab',
    'Lechones + hisopos nasales': 'Mixed',
    'Fluido oral': 'Fluid',
    'Hisopo nasal': 'Swab',
    'Lenguas lechones': 'Tissue',
    'Hisopos nasales': 'Swab',
    'Lenguas': 'Tissue',
    'Hisopos respiratorios': 'Swab',
    'Pulmón, gánglios e hisopos': 'Mixed',
    'Pulmones + hisopos': 'Mixed',
    'Pulmón y gánglios': 'Mixed',
    'Pulmón': 'Tissue',
    'Pulmon + fluidos orales': 'Mixed',
    'Sueros': 'Fluid',
    'Hisopos de vias respiratorias': 'Swab',
    'Lenguas y colas': 'Tissue',
    'Pulmon y ganglio': 'Mixed',
    'Lechón': 'Tissue',
    'Cabeza, cuatro patas y pulmon': 'Mixed',
    'Pulomones': 'Tissue',
    'Fetos': 'Tissue',
    'Feto y placenta': 'Mixed',
    'Fluidos orales y pulmon': 'Mixed',
    'Pulmon e hisopos respiratorios': 'Mixed',
    'Hisopos de saliva': 'Swab',
    'Pulmon con tonsilas': 'Mixed',
    'Pulmon y tonsilas': 'Mixed',
    'Fuidos orales': 'Fluid',
    'Pulmon y lechon': 'Mixed',
    'Fluidos orales + hisopos nasales': 'Mixed',
    'Lenguas y cordones umbilicales': 'Mixed',
    'Pulmnes + 1 lechon': 'Mixed',
    'Pulmon + ganglios': 'Mixed',
    'Hisopo pulmonar': 'Swab',
    'Pulmones + 1 cabeza': 'Mixed',
    'Fluidos orales + hisopos pulmonares + pulmones': 'Mixed',
    'Pulmones + fluidos orales': 'Mixed',
    'Pulmones + 1 fluido oral': 'Mixed',
    'Pulmon, corazon y ganglios': 'Mixed',
    'Pulmones congelados': 'Tissue',
    'Paquetes pulmonares': 'Mixed',
    'Pulmones + 1 hisopo': 'Mixed',
    'Visceras': 'Tissue'
    
}

# Map the SampleType column using the defined mapping
df["SampleType"] = df["SampleType"].map(SampleTypeMapping).fillna("Missing")

FarmTypeMapping = {
    "Granja de madres": "Sow",
    "Cebo" : "Finishing",
    "Cebadero" : "Finishing",
    "Recría" : "Nursery",
    
}

df["PigStages_FarmType"] = df["FarmType"].map(FarmTypeMapping).fillna("Missing")

df["Pathogen"] = df["Pathogen"].map(PathogenMapping).fillna("Missing")


# Step 1: Convert 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')

# Step 2: Format to Year, Month, Day
df['Date'] = df['Date'].dt.strftime("%Y-%m-%d")


In [29]:
df

,SampleNumber,Province,Date,Pathogen,Result,SampleType,Age,FarmType,DiagnosticTest,Country,Breed,Lab_Reference,PigStages_FarmType
0,SV210020,Murcia,2021-01-05,Swine influenza virus,0,Swab,Cerdas abortadas,Granja de madres,PCR,Spain,Meat,3,Sow
1,210031,Badajoz,2021-01-05,Swine influenza virus,0,Tissue,NaN,Cebadero,PCR,Spain,Meat,3,Finishing
2,210071,Girona,2021-01-08,Swine influenza virus,1,Fluid,NaN,Cebadero,PCR,Spain,Meat,3,Finishing
3,210140,Salamanca,2021-01-14,Swine influenza virus,0,Tissue,NaN,Cebadero,PCR,Spain,Meat,3,Finishing
4,210141,Salamanca,2021-01-14,Swine influenza virus,0,Tissue,2.5 meses,Cebadero,PCR,Spain,Meat,3,Finishing
...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,234707,Castellón,2023-10-17,Swine influenza virus,1,Tissue,NaN,Granja de madres,PCR,Spain,Meat,3,Sow
284,SV232530,Murcia,2023-10-18,Swine influenza virus,0,Tissue,Cebo,Cebadero,PCR,Spain,Meat,3,Finishing
285,234780,NaN,2023-10-20,Swine influenza virus,1,Swab,NaN,Cebadero,PCR,Spain,Meat,3,Finishing
286,SV232553,Zamora,2023-10-20,Swine influenza virus,0,Tissue,3.5 - 4 meses,Cebadero,PCR,Spain,Meat,3,Finishing


In [30]:
# Province mapping
ProvinceMapping = {
    'Murcia': 'Murcia',
    'Badajoz': 'Badajoz',
    'Girona ': 'Girona',
    'Salamanca': 'Salamanca',
    'Burgos': 'Burgos',
    'Zamora': 'Zamora',
    'Navarra': 'Navarre',
    'Albacete': 'Albacete',
    'Granada': 'Granada',
    'Lleida': 'Lleida',
    'Las Palmas': 'Las Palmas',
    'Segovia': 'Segovia',
    'Zaragoza': 'Zaragoza',
    'Sevilla': 'Seville',
    'Barcelona': 'Barcelona',
    'Valencia': 'Valencia',
    'Castellón': 'Castellon',
    'Córdoba': 'Cordoba',
    'Málaga': 'Malaga',
    'Ávila': 'Avila',
    'Avila': 'Avila',
    'Teruel': 'Teruel',
    'Cádiz': 'Cadiz',
    'Toledo': 'Toledo',
    'Tarragona': 'Tarragona',
    'Alicante': 'Alicante',
    'Palencia': 'Palencia',
    'Pontevedra': 'Pontevedra',
    'Lugo': 'Lugo',
    'Huesca': 'Huesca',
    'Valladolid': 'Valladolid',
    'Huelva': 'Huelva',
    'La Coruña': 'A Coruna',
    'nan': 'Missing',  # Placeholder for missing values
}
df["Province"] = df["Province"].map(ProvinceMapping).fillna("Missing")



In [31]:
# PigAge mapping
PigStageMapping = {
    'lechones abortados': 'Other',
    'lechon': 'Suckling',  
    'nan': 'Missing',
    '9-10 semanas': 'Nursery',
    '3m0s': 'Finishing',
    '40 kg': 'Finishing',
    '50 kg': 'Finishing',
    'cerda': 'Sow',
    '2,5 meses': 'Finishing',
    '2 Meses': 'Finishing',
    'recria': 'Finishing',
    'recría': 'Finishing',
    's/n': 'Missing',
    'transició': 'Nursery',
    'cebo': 'Finishing',
    'destete': 'Weaning',
    'primerizas destetadas': 'Weaning',
    'primerizas': 'Gilt',
    'semanas 43 y 53': 'Finishing',
    'cerdas gestantes antes parto': 'Sow',
    'cerdo de cebo': 'Finishing',
    '3 meses en cebo': 'Finishing',
    'final transición': 'Nursery',
    '3 semanas en cebo': 'Finishing',
    'fin de cebo-4 meses': 'Finishing',
    '1 mes estancia en cebo': 'Finishing',
    'cerdo 75 kg': 'Finishing',
    '2 meses estancia': 'Finishing',
    '14-18 semanas de vida': 'Nursery',
    '12 semanas de vida': 'Nursery',
    'final destete (6 semanas vida)': 'Nursery',
    '24 semanas de vida': 'Finishing',
    '16 semanas de vida': 'Finishing',
    'multíparas/ primerizas': 'Sow',
    'cerdas gestantes': 'Sow',
    'cerda de recría': 'Gilt',
    'nulíparas/primerizas destetadas': 'Gilt',
    '10 semanas': 'Nursery',
    '1 mes cebo': 'Finishing',
    '3,5 meses cebo': 'Finishing',
    'cerdas gestación': 'Sow',
    'cerdas nulíparas': 'Sow',
    '2 meses cebo': 'Finishing',
    '2,5 meses cebo': 'Finishing',
    '105 kg': 'Finishing',
    'cerda lactante': 'Sow',  # Assuming lactating sow falls under gestation
    '2 meses y medio estancia en cebo': 'Finishing',
    'lechones 3-4 semanas postdestete': 'Nursery',  # Changed to Nursery
    '3 meses de cebo': 'Finishing',
    'primeriza': 'Gilt',
    '11 semanas': 'Nursery',
    '14 semanas': 'Nursery',
    '18 semanas': 'Finishing',
    '20 setmanes': 'Finishing',
    '10 días de vida': 'Suckling',
    'lechones 80 días': 'Finishing',
    # Remaining entries
    'Cerdas abortadas': 'Other',
    '2.5 meses': 'Finishing',
    '15-16 semanas': 'Finishing',
    'Origen madres': 'Sow',
    'Cerda reposición': 'Sow',
    '3 meses': 'Finishing',
    'Cerda': 'Sow',
    '4 meses': 'Finishing',
    '5, 6 y 7 semanas': 'Nursery',
    '6 meses': 'Finishing',
    '10 - 15 Kg': 'Nursery',
    '7 semanas de vida': 'Nursery',
    'Cerdas de recría': 'Gilt',
    'Cerdas': 'Sow',
    '9 y 10 semanas': 'Nursery',
    'Transicion y maternidad': 'Nursery',
    'Cebo': 'Finishing',
    '1 y 4 semanas': 'Suckling',
    'Lechones': 'Suckling',
    'Maternidad y transición': 'Nursery',
    '70 Kg.': 'Finishing',
    '80-90 días de gestación': 'Sow',
    'Lechoneras, transicion, cebo y cerdas': 'Other',
    '3, 6 y 9 semanas': 'Nursery',
    '25 dias de vida': 'Suckling',
    'Lechonera y paridera': 'Suckling',
    'Mitad de cebo': 'Finishing',
    '4 semanas': 'Weaning',  # Changed to Weaning
    '50 dias de vida': 'Suckling',
    '8-10 Kg.': 'Weaning',  # Changed to Weaning
    'Lechones lactacion': 'Suckling',
    'Entrada cebo': 'Finishing',
    'Cebo 40 Kg.': 'Finishing',
    'Lechones de 8 Kg': 'Weaning',  # Changed to Weaning
    'Cerda lactante de 14 meses': 'Sow',
    '30-40 Kg.': 'Finishing',
    '22-25 dias': 'Suckling',
    'Destete': 'Weaning',
    '4-5 semanas de vida': 'Weaning',  # Changed to Weaning
    '20 Kg.': 'Finishing',
    '10-11 semanas': 'Nursery',
    '4 semanas de vida': 'Weaning',  # Changed to Weaning
    '90 Kg': 'Finishing',
    '4 y 10 semanas': 'Nursery',
    '6 semanas de vida': 'Nursery',
    '3 y 5 semanas': 'Weaning',  # Changed to Weaning
    '6 y 9 semanas': 'Nursery',
    '60 Kg.': 'Finishing',
    '50 días': 'Weaning',  # Changed to Weaning
    '5 meses': 'Finishing',
    '48 días': 'Weaning',  # Changed to Weaning
    '70 días': 'Finishing',
    'Cerdas Primerizas y Multíparas': 'Sow',
    '4, 6 y 9 semanas': 'Nursery',
    '6-7 semanas de vida': 'Nursery',
    '10 semanas de vida': 'Finishing',
    '5 y 7 semanas': 'Nursery',
}

# Map the 'Edad' column using the English translation mapping
df['PigStages_Age'] = df['Age'].str.lower().map(PigStageMapping)

# Combine the results from both mappings (PigStages_Age and PigStages_FarmType)
#df['PigStages'] = df.apply(lambda x: x['PigStages_Age'] if pd.notna(x['PigStages_Age']) else x['PigStages_FarmType'], axis=1)
df['PigStage'] = df.apply(lambda x: x['PigStages_Age'] if pd.notna(x['PigStages_Age']) else x['PigStages_FarmType'], axis=1).fillna("Missing")

# Drop intermediate columns if needed
df.drop(['PigStages_Age', 'PigStages_FarmType'], axis=1, inplace=True)

df = df.drop(['Age'], axis=1)
df = df.drop(['FarmType'], axis=1)

In [32]:
df.head(50)

,SampleNumber,Province,Date,Pathogen,Result,SampleType,DiagnosticTest,Country,Breed,Lab_Reference,PigStage
0,SV210020,Murcia,2021-01-05,Swine influenza virus,0,Swab,PCR,Spain,Meat,3,Sow
1,210031,Badajoz,2021-01-05,Swine influenza virus,0,Tissue,PCR,Spain,Meat,3,Finishing
2,210071,Girona,2021-01-08,Swine influenza virus,1,Fluid,PCR,Spain,Meat,3,Finishing
3,210140,Salamanca,2021-01-14,Swine influenza virus,0,Tissue,PCR,Spain,Meat,3,Finishing
4,210141,Salamanca,2021-01-14,Swine influenza virus,0,Tissue,PCR,Spain,Meat,3,Finishing
5,SV210121,Burgos,2021-01-19,Swine influenza virus,0,Tissue,PCR,Spain,Meat,3,Finishing
6,SV210179,Zamora,2021-10-21,Swine influenza virus,0,Swab,PCR,Spain,Meat,3,Sow
7,SV210233,Navarre,2021-01-26,Swine influenza virus,1,Tissue,PCR,Spain,Meat,3,Finishing
8,SV210252,Albacete,2021-01-27,Swine influenza virus,0,Tissue,PCR,Spain,Meat,3,Finishing
9,SV210330,Missing,2021-02-03,Swine influenza virus,1,Tissue,PCR,Spain,Meat,3,Nursery


In [33]:
df.head(300)

,SampleNumber,Province,Date,Pathogen,Result,SampleType,DiagnosticTest,Country,Breed,Lab_Reference,PigStage
0,SV210020,Murcia,2021-01-05,Swine influenza virus,0,Swab,PCR,Spain,Meat,3,Sow
1,210031,Badajoz,2021-01-05,Swine influenza virus,0,Tissue,PCR,Spain,Meat,3,Finishing
2,210071,Girona,2021-01-08,Swine influenza virus,1,Fluid,PCR,Spain,Meat,3,Finishing
3,210140,Salamanca,2021-01-14,Swine influenza virus,0,Tissue,PCR,Spain,Meat,3,Finishing
4,210141,Salamanca,2021-01-14,Swine influenza virus,0,Tissue,PCR,Spain,Meat,3,Finishing
...,...,...,...,...,...,...,...,...,...,...,...
283,234707,Castellon,2023-10-17,Swine influenza virus,1,Tissue,PCR,Spain,Meat,3,Sow
284,SV232530,Murcia,2023-10-18,Swine influenza virus,0,Tissue,PCR,Spain,Meat,3,Finishing
285,234780,Missing,2023-10-20,Swine influenza virus,1,Swab,PCR,Spain,Meat,3,Finishing
286,SV232553,Zamora,2023-10-20,Swine influenza virus,0,Tissue,PCR,Spain,Meat,3,Finishing


In [34]:
# Save to desired path
df.to_csv('output/PigLab3.csv', index=False)

In [35]:
barometer = df

In [36]:
import pandas as pd

# Load the dataset
file_path = "data/PigData/barometer_combined_2024-10-08.csv"
df = pd.read_csv(file_path)

# Filter data for LabReference 1 and assign to 'barometer'
barometer = df[df['Lab_Reference'] == 3]

# Drop unwanted columns
barometer = barometer.drop(columns=['Month', 'Year', 'AutonomousCommunity'])

barometer['Pathogen'] = barometer['Pathogen'].str.strip().str.replace(' ', '_', regex=False)


# Rename 'FlooredDate' to 'Date'
barometer = barometer.rename(columns={'FlooredDate': 'Date'})

# Display the cleaned DataFrame
barometer.head()

# Optionally, save to CSV
barometer.to_csv("barometer.csv", index=False)



In [37]:
barometer

,SampleNumber,Province,Date,Pathogen,Result,SampleType,DiagnosticTest,Country,Breed,Lab_Reference,PigStage
1512,SV210020,Murcia,2021-01-05,Swine_influenza_virus,0,Swab,PCR,Spain,Meat,3,Sow
1513,210031,Badajoz,2021-01-05,Swine_influenza_virus,0,Tissue,PCR,Spain,Meat,3,Finishing
1514,210071,Girona,2021-01-08,Swine_influenza_virus,1,Fluid,PCR,Spain,Meat,3,Finishing
1515,210140,Salamanca,2021-01-14,Swine_influenza_virus,0,Tissue,PCR,Spain,Meat,3,Finishing
1516,210141,Salamanca,2021-01-14,Swine_influenza_virus,0,Tissue,PCR,Spain,Meat,3,Finishing
...,...,...,...,...,...,...,...,...,...,...,...
3048,SV232914,Madrid,2023-11-30,Lawsonia_intracellularis,1,Feces,PCR,Spain,Meat,3,Finishing
3049,SV232928,Salamanca,2023-12-04,Lawsonia_intracellularis,0,Feces,PCR,Spain,Meat,3,Suckling
3050,235796,Salamanca,2023-12-05,Lawsonia_intracellularis,1,Swab,PCR,Spain,Meat,3,Finishing
3051,SV232951,Segovia,2023-12-05,Lawsonia_intracellularis,1,Swab,PCR,Spain,Meat,3,Nursery


### Step 02: Create an RDF graph and namespaces.

In [38]:
g = rdflib.Graph()
xsd = Namespace('http://www.w3.org/2001/XMLSchema#')
g.bind('xsd', xsd)

# Define your custom namespace for your ontology's properties
LHO = Namespace("http://www.purl.org/decide/LiveStockHealthOnto/LHO#")
g.bind('LHO', LHO)

SKOS = Namespace("http://www.w3.org/2004/02/skos/core#")
g.bind('skos', SKOS)

decide=  Namespace("http://www.purl.org/decide#")
g.bind('decide', decide)
ncit = Namespace("http://purl.obolibrary.org/obo/NCIT_C25464")
g.bind('ncit', ncit)
agrovoc = Namespace ("http://aims.fao.org/aos/agrovoc")
g.bind('agrovoc', agrovoc)

###  Step 03: Iterate over the Panda DataFrame and map to ontology properties:

In [39]:
#For Pig Data frame RDF conversion
for index, row in barometer.iterrows():
    SampleNumber = row["SampleNumber"]
    Date = row["Date"]
    Pathogen = row["Pathogen"]
    Breed = row["Breed"]
    Country = row["Country"]
    Province = row["Province"]
    SampleType = row["SampleType"]
    SampleResult = row["Result"]
    DiagnosticTest = row["DiagnosticTest"]
    PigStage= row["PigStage"]

    
    # Create a unique URI for each sample based on the row index
    PigSample_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#Lab3PigSample_{index}")
    
    # Add sample type assertion
    g.add((PigSample_uri, RDF.type, LHO.PigSample))
    description = "An individual representing a sample from a Pig and Piglets."
    g.add((PigSample_uri, RDFS.comment, Literal(description, lang="en")))
    
    if "Pathogen" in row and row["Pathogen"]:
        Pathogen = row["Pathogen"]
        Pathogen_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Pathogen}")
        g.add((PigSample_uri, LHO.hasPathogen, Pathogen_uri))
        g.add((Pathogen_uri, RDF.type, decide.Pathogen))
        description = "An individual representing Pig pathogen."
        g.add((Pathogen_uri, RDFS.comment, Literal(description, lang="en")))
        
    if "Breed" in row and row["Breed"]:
        Breed = row["Breed"]
        Breed_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Breed}")
        g.add((PigSample_uri, LHO.hasBreed, Breed_uri))
        g.add((Breed_uri, RDF.type, LHO.Breed))
        description = "An individual representing a Piglet breed which is Meat."
        g.add((Breed_uri, RDFS.comment, Literal(description, lang="en")))
        
    if "DiagnosticTest" in row and row["DiagnosticTest"]:
        DiagnosticTest= row["DiagnosticTest"]
        DiagnosticTest_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{DiagnosticTest}")
        g.add((PigSample_uri, LHO.hasDiagnosticTest, DiagnosticTest_uri))
        g.add((DiagnosticTest_uri, RDF.type, LHO.DiagnosticTest))
    if DiagnosticTest == "PCR":
        description = "An individual representing DNA/RNA amplification for rapid pathogen detection."
    elif SampleType == "Culture":
        description = "An individual representing a Growing live microorganisms for identification."
        g.add((DiagnosticTest_uri, RDFS.comment, Literal(description, lang="en")))
        
    if "Country" in row and row["Country"]:
        Country = row["Country"].strip()  # Use strip() to remove leading and trailing spaces
        # Encode the Country value to create a valid URI
        Country = urllib.parse.quote(Country)
        Country_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Country}")
    
        # Map to the specific class URI in your ontology and add the label
        g.add((PigSample_uri, LHO.hasCountry, Country_uri))
        g.add((Country_uri, RDF.type, URIRef("http://purl.obolibrary.org/obo/NCIT_C25464")))  # Use the specific class URI
        g.add((Country_uri, RDFS.label, Literal("Spain")))
        description = "An individual representing different Countries."
        g.add((Country_uri, RDFS.comment, Literal(description, lang="en")))
        
    if "Province" in row and row["Province"]:
        Province = row["Province"].strip()
        Province = urllib.parse.quote(Province)
        Province_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Province}")
        g.add((PigSample_uri, LHO.hasProvince, Province_uri))
        g.add((Province_uri, RDF.type, LHO.Province))
        description = "An individual representing different Province."
        g.add((Province_uri, RDFS.comment, Literal(description, lang="en")))
    
    if "PigStage" in row and row["PigStage"]:
        Stage = row["PigStage"]
        Stage_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Stage}")
        g.add((PigSample_uri, LHO.hasProductionStages, Stage_uri))
        g.add((Stage_uri, RDF.type, LHO.PigProductionStage))
        description = "An individual representing Production stage ."
        g.add((Stage_uri, RDFS.comment, Literal(description, lang="en")))
    
    if "SampleType" in row and row["SampleType"]:
        SampleType = row["SampleType"]
        SampleType_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{SampleType}")
        # Add statements for SampleType
        g.add((PigSample_uri, LHO.hasSampleType, SampleType_uri))
        g.add((SampleType_uri, RDF.type, URIRef("http://www.purl.org/decide#SampleType")))
    if SampleType == "Autopsy":
        description = "An individual representing a cattle sample obtained through autopsy."
    elif SampleType == "BAL":
        description = "An individual representing a cattle sample obtained through bronchoalveolar lavage (BAL)."
    elif SampleType == "SWAB":
        description = "An individual representing a cattle sample obtained through swabbing."
    elif SampleType == "Missing":
        description = "An individual representing a missing or unspecified cattle sample type."
        g.add((SampleType_uri, RDFS.comment, Literal(description, lang="en")))
           
    if "Result" in row and row["Result"]:
        SampleResult = row["Result"]
        # Create a unique URI for the Result based on the value
        Result_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{SampleResult}")
        g.add((PigSample_uri, LHO.hasResult, Result_uri))
        g.add((Result_uri, RDF.type, decide.SampleResult))
    if SampleResult == "1":
        description = "An individual representing a Positive test result"
    elif SampleResult == "0":
        description = "An individual representing a negative test result"
    elif SampleResult == "missing":
        description = "An individual representing an unknown or missing test result."
    g.add((Result_uri, RDFS.comment, Literal(description, lang="en")))


    if "Date" in row and not pd.isnull(row["Date"]):  # Check for NaN or None values
        Date = str(row["Date"]).strip()
    
    # Split the datetime string and take the date part
        Date = Date.split()[0]
    
    # Create a unique URI for the Date based on the value (use the Date directly)
        Date_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Date}")
    
    # Add Date assertion
        g.add((PigSample_uri, LHO.hasDate, Date_uri))
        g.add((Date_uri, RDF.type, LHO.Date))
    

# Serialize the RDF graph to a file
rdf_output_file = "output/PigLab3.rdf"
g.serialize(rdf_output_file, format="xml")

# Serialize the RDF graph to Turtle format and print it too see RDF conversion
turtle_data = g.serialize(format="turtle")
print(turtle_data)

@prefix LHO: <http://www.purl.org/decide/LiveStockHealthOnto/LHO#> .
@prefix decide: <http://www.purl.org/decide#> .
@prefix ncit: <http://purl.obolibrary.org/obo/NCIT_C25464> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

LHO:Lab3PigSample_1512 a LHO:PigSample ;
    LHO:hasBreed LHO:Meat ;
    LHO:hasCountry LHO:Spain ;
    LHO:hasDate LHO:2021-01-05 ;
    LHO:hasDiagnosticTest LHO:PCR ;
    LHO:hasPathogen LHO:Swine_influenza_virus ;
    LHO:hasProductionStages LHO:Sow ;
    LHO:hasProvince LHO:Murcia ;
    LHO:hasResult LHO:0 ;
    LHO:hasSampleType LHO:Swab ;
    rdfs:comment "An individual representing a sample from a Pig and Piglets."@en .

LHO:Lab3PigSample_1513 a LHO:PigSample ;
    LHO:hasBreed LHO:Meat ;
    LHO:hasCountry LHO:Spain ;
    LHO:hasDate LHO:2021-01-05 ;
    LHO:hasDiagnosticTest LHO:PCR ;
    LHO:hasPathogen LHO:Swine_influenza_virus ;
    LHO:hasProductionStages LHO:Finishing ;
    LHO:hasProvince LHO:Badajoz ;
    LHO:hasResult LHO:0 ;
    LHO:hasS

### Step 4: For Knowledge Graph,Load the RDF data and ontology into a Panda DataFrame: 

In [40]:
# Parse the ontology file in OWL format and add it to the graph
path_to_ontology = "Ontology/LivestockHealthOntology.owl"
g.parse(path_to_ontology, format="xml")

<Graph identifier=N78176770b7064649abfd3a5684c5a73a (<class 'rdflib.graph.Graph'>)>

### Step 05: Query the data from updated ontology 

In [41]:

query = """
PREFIX decide: <http://www.purl.org/decide#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX LHO: <http://www.purl.org/decide/LiveStockHealthOnto/LHO#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT
  (strafter(str(?Sample), "#") AS ?SampleName)
  (strafter(str(?Pathogen), "#") AS ?PathogenName)
  (strafter(str(?Breed), "#") AS ?BreedName)
  (strafter(str(?SampleType), "#") AS ?SampleTypeName)
  (strafter(str(?DiagnosticTest), "#") AS ?DiagnosticTestName)
  (strafter(str(?PigProductionStages), "#") AS ?ProductionStage)
  (strafter(str(?Province), "#") AS ?ProvinceName)
  (strafter(str(?Country), "#") AS ?CountryName)
  (strafter(str(?SampleResult), "#") AS ?SampleResultName)

WHERE {
 {
  ?Sample rdf:type LHO:PigSample .
  ?Sample LHO:hasPathogen ?Pathogen .
  ?Sample LHO:hasBreed ?Breed .
  ?Sample LHO:hasSampleType ?SampleType .
  FILTER (?SampleType = LHO:Fluid)
  ?Sample LHO:hasProvince ?Province .
  ?Sample LHO:hasCountry ?Country .
  ?Sample LHO:hasProductionStages ?PigProductionStages .
  FILTER (?PigProductionStages = LHO:Weaning)
  ?Sample LHO:hasResult ?SampleResult .
  FILTER (?SampleResult = LHO:1)
  ?Sample LHO:hasDiagnosticTest ?DiagnosticTest .
  }
}
"""


# Execute the query and retrieve the results
results = g.query(query)

# Convert the results to a Pandas dataframe
data = []
for row in results:
    data.append(list(row))

df = pd.DataFrame(data, columns=[
    "Sample", "Pathogen", "Breed", "SampleType", 
    "DiagnosticTest","PigProductionStage", "Province", "Country", "SampleResult"
])

# Display the dataframe
df


,Sample,Pathogen,Breed,SampleType,DiagnosticTest,PigProductionStage,Province,Country,SampleResult
0,Lab3PigSample_1668,Swine_influenza_virus,Meat,Fluid,PCR,Weaning,Girona,Spain,1
1,Lab3PigSample_1707,Swine_influenza_virus,Meat,Fluid,PCR,Weaning,Castellon,Spain,1
